![](https://media.licdn.com/dms/image/v2/D5622AQEQJ3u_ECwEzA/feedshare-shrink_2048_1536/B56Zuz4Cy3HcAk-/0/1768249372070?e=1770249600&v=beta&t=mas2waJIQx2xRTIxJUlK_55097wUCjR1yrL30W_WMZ0)

In [0]:
%python
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
from pyspark.sql.functions import to_timestamp

data_dict = [
{"txn_id":1,"vehicle_no":"DL01AA1111","vehicle_type":"car","crossing_time":"2026-01-10 08:00"},
{"txn_id":2,"vehicle_no":"DL01AA1111","vehicle_type":"car","crossing_time":"2026-01-10 11:00"},
{"txn_id":3,"vehicle_no":"DL02BB2222","vehicle_type":"truck","crossing_time":"2026-01-10 09:00"},
{"txn_id":4,"vehicle_no":"DL02BB2222","vehicle_type":"truck","crossing_time":"2026-01-10 16:00"},
{"txn_id":5,"vehicle_no":"DL03CC3333","vehicle_type":"bus","crossing_time":"2026-01-10 10:00"},
{"txn_id":6,"vehicle_no":"DL03CC3333","vehicle_type":"bus","crossing_time":"2026-01-10 12:00"},
{"txn_id":7,"vehicle_no":"DL04DD4444","vehicle_type":"motorcycle","crossing_time":"2026-01-10 11:00"},
{"txn_id":8,"vehicle_no":"DL05EE5555","vehicle_type":"car","crossing_time":"2026-01-10 14:00"}
]
schema1 = StructType([
    StructField("txn_id", IntegerType(), False),
    StructField("vehicle_no", StringType(), False),
    StructField("vehicle_type", StringType(), False),
    StructField("crossing_time", StringType(), False)
])
df1 = spark.createDataFrame(data_dict, schema=schema1)
df2=df1.withColumn("crossing_time", to_timestamp("crossing_time"))


In [0]:
%python
from pyspark.sql.window import Window
from pyspark.sql.functions import lag,timestamp_diff,when, isnull,col,sum
df3=df2.withColumn("prev_crossing_time",lag("crossing_time",1).over(Window.partitionBy("vehicle_no").orderBy("crossing_time"))).withColumn("time_dff",timestamp_diff("minute","prev_crossing_time","crossing_time"))
df4 = df3.selectExpr(
    "*",
    """
    CASE
        WHEN vehicle_type = 'car'
             AND time_dff <= 240 THEN 20
        WHEN vehicle_type = 'car'
             AND (time_dff > 240 OR time_dff IS NULL) THEN 40

        WHEN vehicle_type = 'truck'
             AND time_dff <= 240 THEN 40
        WHEN vehicle_type = 'truck'
             AND (time_dff > 240 OR time_dff IS NULL) THEN 80

        WHEN vehicle_type = 'bus'
             AND time_dff <= 240 THEN 30
        WHEN vehicle_type = 'bus'
             AND (time_dff > 240 OR time_dff IS NULL) THEN 70
    END AS amount
    """
)
display(df4.groupBy().sum("amount"))

    
display(df4)
